In [ ]:
### Notebook used to test & compare outputs between MATLAB & Python implementations
## Run cell by cell, some cells N/A for certain methods

In [1]:
import sys
from scipy import fft
from scipy.fft import fftshift
import numpy as np
from multiprocessing import Pool
from scipy.stats import pearsonr
import Bio
from Bio.Phylo.TreeConstruction import *
from functools import partial

import pywt
# import os

from statistics import median, mean
from one_dimensional_num_mapping import *

# plotting

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from classification import classify_dismat
from preprocessing import preprocessing
from helpers import *

from cgr import *
from visualisation import dimReduction
# set up
# if os.path.exists("Sequence_database.idx"):
#     os.remove("Sequence_database.idx")


In [2]:
#All validation tests run with Primates dataset
data_set = './DataBase/Primates'


In [13]:
test_set = 'NoData'
seq_to_test = 0
min_seq_len = 0
max_clust_size = 5000000
frags_per_seq = 1

methods_list = {0: cgr, 1: num_mapping_PP, 2: num_mapping_Int, 3: num_mapping_IntN, 5: num_mapping_Doublet, 6: num_mapping_Codons, 7: num_mapping_Atomic,
                8: num_mapping_EIIP, 9: num_mapping_AT_CG, 10: num_mapping_justA, 11: num_mapping_justC, 12: num_mapping_justG, 13: num_mapping_justT, 14: 'PuPyCGR', 15: '1DPuPyCGR'}


In [4]:
def compute_cgr_PuPyCGR(seq_index):
    # seqs is the sequence database, it is being called by accession number
    # (keys) which is being iterated over all seq_index,
    # (count of all sequences in uploaded dataset) .seq calls the string
    seq = str(seq_dict[keys[seq_index]].seq)
    seq_new = seq
    if method_num == 14:
        seq_new = seq_new.replace('G', 'A')
        seq_new = seq_new.replace('C', 'T')
    cgr_output = cgr(seq_new, 'ACGT', k_val)  # shape:[2^k, 2^k]
    # shape:[2^k, 2^k] # may not be appropriate to take by column
    # axis=0 takes fft by column, consistent with MATLAB 
    fft_output = fft.fft(cgr_output,axis=0)
    # order='F' flattens the array in fortran index consistent with MATLAB, shape: [1,2^k*2^k], list makes sorting easy for comparison of distance matrices
    abs_fft_output = np.abs(fft_output.flatten(order='F'))
    return abs_fft_output, fft_output, cgr_output, seq_new, seq  # flatted into 1d array


In [5]:
def compute_1DPuPyCGR(seq_index):
    seq = str(seq_dict[keys[seq_index]].seq)
    # creates PuPyCGR
    seq_new = seq.replace('G', 'A')
    seq_new = seq_new.replace('C', 'T')
    cgr_raw = cgr(seq_new, 'ACGT', k_val)
    # takes only the last (bottom) row but all columns of cgr to make 1DPuPyCGR
    cgr_output = cgr_raw[-1, :]
    # shape:[1, 2^k] # may not be appropriate to take by column
    fft_output = fft.fft(cgr_output)
    abs_fft_output = np.abs(fft_output.flatten(order='F'))
    return abs_fft_output, fft_output, cgr_output, seq_new, seq # flatted into 1d array


In [6]:
def one_dimensional_num_mapping_wrapper(seq_index):
    # normalize sequences to median sequence length of cluster
    seq = str(seq_dict[keys[seq_index]].seq)
    if len(seq) >= med_len:
        seq_new = seq[0:round(med_len)]
    else:
        seq_new=seq
    num_seq = method(seq_new)
    if len(num_seq) < med_len:
        pad_width = int(med_len - len(num_seq))
        num_seq = pywt.pad(num_seq, pad_width, 'antisymmetric')[pad_width:]
    fft_output = fft.fft(num_seq)
    abs_fft_output = np.abs(fft_output.flatten()) #order='F' for testing abs_fft_outputs vs matlab
    return abs_fft_output, fft_output, num_seq, seq


In [7]:
def compute_pearson_coeffient(x, y):
    r = pearsonr(x, y)[0]
    normalized_r = (1-r)/2
    return normalized_r

def compute_pearson_coeffient_wrapper(i, j, abs_fft_output_list):
    # print(abs_fft_output_list)
    # x = abs_fft_output_list[i]
    # y = abs_fft_output_list[indices[1]]
    # print(x)
    # print(y)
    return compute_pearson_coeffient(abs_fft_output_list[i], abs_fft_output_list[j])


In [17]:
# change method number referring the variable above (between 0 and 15)
from IPython.display import clear_output
for f in methods_list:
    method_num = f
    print(method_num)
    method = methods_list[f]
    if method_num in range(0,14):
        method_name = method.__name__
    else:
        method_name= method
    print(method_name)
    file = open(f"{method_name}_validation.txt", "x")
    # all matlab test .mat files were run at k=4
    k_val = 4  # used only for CGR-based representations(if methodNum=1,15,16)
    seq_dict, number_of_clusters, total_seq, cluster_dict = preprocessing(data_set,max_clust_size)
    keys = list(cluster_dict.keys())
    values = list(cluster_dict.values())
    # Could be parallelized in the future
    seqs_length = [len(seq_dict[keys[i]].seq) for i in range(total_seq)]
    med_len = median(seqs_length)
    labels = [cluster_dict[x] for x in seq_dict.keys()]
    fft_output_list = []
    abs_fft_output_list = []
    cgr_output_list = []
    seq_new_list = []
    seq_list=[]
    print('Generating numerical sequences, applying DFT, computing magnitude spectra .... \n')
    for seq_index in range(total_seq):
        if method_num == 0 or method_num == 14:
            abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_cgr_PuPyCGR(seq_index)
            abs_fft_output_list.append(abs_fft_output)
            fft_output_list.append(fft_output)
            cgr_output_list.append(cgr_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)
            
        elif method_num == 15:
            abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_1DPuPyCGR(seq_index)
            abs_fft_output_list.append(abs_fft_output)
            fft_output_list.append(fft_output)
            cgr_output_list.append(cgr_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)

        else:
            abs_fft_output, fft_output, seq_new, seq = one_dimensional_num_mapping_wrapper(seq_index)
            fft_output_list.append(fft_output)
            abs_fft_output_list.append(abs_fft_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)
    
    print('Building distance matrix')
    distance_matrix = []
    for i in range(total_seq):
        for j in range(total_seq):
            distance_matrix.append(compute_pearson_coeffient_wrapper(i,j,abs_fft_output_list)) 
    distance_matrix = np.array(distance_matrix).reshape(total_seq, total_seq)
    # np.savetxt('distmat.txt', distance_matrix)

    print('Performing classification .... \n')
    folds = 10
    if (total_seq < folds):
        folds = total_seq
    mean_accuracy, accuracies, confusion_matrix, misclassified_id = classify_dismat(distance_matrix, labels, folds, total_seq)
    # accuracy,avg_accuracy, clNames, cMat
    # accuracies = [accuracy, avg_accuracy];
    print('Classification accuracy 5 classifiers:\n', accuracies, file=file)
    print('**** Processing completed ****\n')


    ### VALIDATION START ###
    from scipy.io import loadmat
    #load .mat file from your machine
    matlab = loadmat(f'./DataBase/primates_{method_name}.mat',simplify_cells=True)
    locals().update(matlab)
    # compare raw input sequences
    print(f"\nComparing raw input sequences:\n{seq_list==Seq}", file=file)

    # compare output sequences for cgr methods; 
    if method_num in [0,14,15]:
        print(f"\nCompare cgr inputs, only different for non-standard cgr:\n {seq_new_list==sequence}", file=file)
        print("\nComparing each sample's cgr output", file=file)
        for thing in range(total_seq):
            print(True==np.allclose(cgr_output_list[thing],nmValSH[thing]),file=file)
    else:
        print("\nCompare output of numerical representation:",file=file)
        for thing in range(total_seq):
            print(seq_new_list[thing]==nmValSH[thing],file=file)
    
    print("\nCompare fft outputs:",file=file)
    for thing in range(total_seq):
        print(np.allclose(fft_output_list[thing],f[thing]),file=file)
    
    print("\nCompare magnitudes of ffts:",file=file)
    for thing in range(total_seq):
        print(np.allclose(abs_fft_output_list[thing],lg[thing]),file=file)
    
    print(f"\nCompare distance matrix:\n{np.allclose(distance_matrix,disMat,atol=1e-15,rtol=1e-11)}",file=file)
    file.close()
    clear_output()
    # objects = dir()
    # for obj in objects:
    #     if not obj.startswith("__"):
    #         del globals()[obj]

13
<function num_mapping_justT at 0x12cdb34c0>
Generating numerical sequences, applying DFT, computing magnitude spectra .... 

Building distance matrix
Performing classification .... 

**** Processing completed ****



FileNotFoundError: [Errno 2] No such file or directory: './DataBase/primates_<function num_mapping_justT at 0x12cdb34c0>.mat'

In [5]:
seq_dict, number_of_clusters, total_seq, cluster_dict = preprocessing(data_set,max_clust_size)


In [6]:
# variable holding all the keys (accession numbers) for corresponding clusters

keys = list(cluster_dict.keys())

values = list(cluster_dict.values())
# Could be parallelized in the future

seqs_length = [len(seq_dict[keys[i]].seq) for i in range(total_seq)]
med_len = median(seqs_length)
labels = [cluster_dict[x] for x in seq_dict.keys()]
fft_output_list = []
abs_fft_output_list = []
cgr_output_list = []
seq_new_list = []
seq_list=[]


In [ ]:
# #Phylogenetic trees not compared
# def phylogenetic_tree(distance_matrix):
#     names = keys
#     matrix = triangle_matrix
#     distance_matrix = DistanceMatrix(names, matrix)
#     constructor = DistanceTreeConstructor()
#     nj_tree = constructor.nj(distance_matrix)
#     # newick may not be need to be quoted
#     neighbour_joining_tree = nj_tree.format('newick')
#     upgma = constructor.upgma(distance_matrix)
#     upgma_tree = upgma.format('newick')
#     print(upgma_tree, file=tree_print)
#     # can add code here for visualization with matplotlib


In [12]:
print('Generating numerical sequences, applying DFT, computing magnitude spectra .... \n')
for seq_index in range(total_seq):
    if method_num == 0 or method_num == 14:
        abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_cgr_PuPyCGR(seq_index)
        abs_fft_output_list.append(abs_fft_output)
        fft_output_list.append(fft_output)
        cgr_output_list.append(cgr_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)
        
    elif method_num == 15:
        abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_1DPuPyCGR(seq_index)
        abs_fft_output_list.append(abs_fft_output)
        fft_output_list.append(fft_output)
        cgr_output_list.append(cgr_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)

    else:
        abs_fft_output, fft_output, seq_new, seq = one_dimensional_num_mapping_wrapper(seq_index)
        fft_output_list.append(fft_output)
        abs_fft_output_list.append(abs_fft_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)

Generating numerical sequences, applying DFT, computing magnitude spectra .... 



In [13]:
print('Building distance matrix')
distance_matrix = []
for i in range(total_seq):
    for j in range(total_seq):
        distance_matrix.append(compute_pearson_coeffient_wrapper(i,j,abs_fft_output_list)) 
distance_matrix = np.array(distance_matrix).reshape(total_seq, total_seq)
# np.savetxt('distmat.txt', distance_matrix)


Building distance matrix


In [ ]:
    # print('Building Phylogenetic Trees')
    # matrix_list = distance_matrix.tolist()
    # triangle_matrix = []
    # # loop to create triangle matrix as nested list
    # for i in range(total_seq):
    #     row = []
    #     row = (matrix_list[i][0:i+1])
    #     triangle_matrix.append(row)
    # # change filename to unique ID
    # tree_print = open('upgma.tree', 'a')
    # phylogenetic_tree(triangle_matrix)
    # tree_print.close()

In [16]:
    print('Performing classification .... \n')
    folds = 10
    if (total_seq < folds):
        folds = total_seq
    mean_accuracy, accuracies, confusion_matrix, misclassified_id = classify_dismat(distance_matrix, labels, folds, total_seq)
    # accuracy,avg_accuracy, clNames, cMat
    # accuracies = [accuracy, avg_accuracy];
    print('Classification accuracy 5 classifiers\n', accuracies)
    print('**** Processing completed ****\n')

Performing classification .... 

Classification accuracy 5 classifiers
 {'LinearDiscriminant': 0.9061904761904762, 'LinearSVM': 0.9657142857142856, 'QuadSVM': 0.8385714285714286, 'KNN': 0.9257142857142858}
**** Processing completed ****



In [33]:
# Save entire matlab workspace as .mat 
# Use matlab mldsp version from github as variables have been added for testing; make sure k_val (where applicable) and methods match
from scipy.io import loadmat
#load .mat file from your machine
matlab = loadmat('./DataBase/primates_cgr.mat',simplify_cells=True)
locals().update(matlab)

/Users/dolteanu/.pyenv/versions/MLDSP/lib/python3.8/site-packages/scipy/io/matlab/mio.py:224: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [34]:
# Testing convention in this notebook: python variable on the left, matlab variable on the right
# compare accession number
keys==AcNmb

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [35]:
#compare raw input sequences
seq_list==Seq

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [36]:
#Run only for cgr based methods
# compare output sequences for cgr methods; regular cgr will be the same as previous cell since raw sequence is used for cgr generation
seq_new_list==sequence

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
# Run only for 1D numerical representation methods
# Compare output of numerical representation
for thing in range(total_seq):
    print(seq_new_list[thing]==nmValSH[thing])

In [37]:
# Run only for cgr methods
# compare CGRs
for thing in range(total_seq):
    print(True==np.allclose(cgr_output_list[thing],nmValSH[thing]))
        

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [38]:
# Compare fft outputs
for thing in range(total_seq):
    print(np.allclose(fft_output_list[thing],f[thing]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [39]:
# Compare magnitudes of ffts
for thing in range(total_seq):
    print(np.allclose(abs_fft_output_list[thing],lg[thing]))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [40]:
#Compare distance matrices
#modify exponent of 'rtol' to find similarity of dist mats, atol should be left untouched as it tests precision of zeros (diagonal)
np.allclose(distance_matrix,disMat,atol=1e-15,rtol=1e-11)

True

In [41]:
#Compare MDS outputs-currently not matching
scaled_distance_matrix = dimReduction(distance_matrix, n_dim=3, method='pca')
np.isclose(scaled_distance_matrix,Y,rtol=1e0)

array([[False, False, False],
       [False, False, False],
       [False,  True, False],
       [False, False,  True],
       [False,  True, False],
       [False, False,  True],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [False, False,  True],
       [False, False,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False,  True, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [Fa